<a href="https://colab.research.google.com/github/isabela2423/taller-1/blob/main/Taller_2_ISABELA_HERN%C3%81NDEZ_Y_SEBASTIAN_ALFARO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Taller #2
Integrantes: Sebastian Alfaro e Isabela Hernández



In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.optimize import linprog
import cvxpy as cp

PUNTO 1. La empresa XYZ produce dos productos: A y B. La ganancia por unidad de A es de $3
dólares y de B es de $5 dólares. Para su producción dispone de 500 unidades de materia
prima, 1000 unidades de mano de obra y 2000 dólares de capital, de los cuales se
necesitan para producir una unidad de:
i) A: una unidad de materia prima, dos unidades de mano de obra y una unidad de capital.
ii) B: una unidad de materia prima, cuatro unidades de mano de obra y dos unidades de
capital.
Además, se necesita producir al menos 100 unidades de A y 150 unidades de B para
satisfacer la demanda del mercado.
Encuentre la asignación óptima de A y B que maximizan las ganancias de la empresa.
Para ello, formule un problema de LP utilizando Scipy.

In [ ]:

c = np.array([-3, -5])
A = np.array([[1, 1], [2, 4], [1, 2]])
b = np.array([500, 1000, 2000])
bounds = ((100, None), (150, None))

res = linprog(c=c, A_ub=A, b_ub=b, bounds=bounds)

print('Ganancias máximas: $' + str(-res.fun))

Ganancias máximas: $1350.0


In [ ]:
# Interpretación:Si la empresa produce los productos A y B, mediante las restricciones dadas las ganancias máximas de la empresa serían de 1350.
# Las cantidades específicas de A deben ser de 200 y de B deben ser de 150
# para que se maximice la ganancia

PUNTO 2. Usted desea invertir la suma de $100 millones en cuatro fondos que tienen diferentes
retornos esperados (E(R)), así como diferentes ponderaciones en acciones de alto valor,
gran tamaño y alta liquidez, como muestra la siguiente tabla:



La asignación anterior debe contener al menos un 35% de acciones de alto valor, un 30%
en tamaño y un 15% de acciones de liquidez. Encuentre una asignación factible con el
retorno esperado más alto, bajo el supuesto de que solo se le permite mantener posiciones
largas en los fondos.
Formule y resuelva el problema de LP considerando las siguientes restricciones:

0.5 x1 + 0.3 x2 + 0.25 x3 + 0.6 x4 ≥ 0.35 (valor)
0.3 x1 + 0.1 x2 + 0.4 x3 + 0.2 x4 ≥ 0.30 (tamaño)
0.2 x1 + 0.6 x2 + 0.35 x3 + 0.2 x4 ≥ 0.15 (liquidez)

x1 + x2 + x3 + x4 = $100
x1, x2, x3, x4 ≥ 0

Nota: resuelva el problema anterior usando las librerías de Scipy y CVXPY de Python.

In [ ]:
#LIBRERIA SCIPY
# Parametros de la función objetivo
fun_objt = np.array([0.10 , 0.15 ,0.16 ,0.08])
# Restricciones de desigualdad
res1 = np.array([[0.5,0.3,0.25,0.6],[0.3,0.1,0.4,0.2],[0.2,0.6,0.35,0.2]])
res2= np.array([0.35,0.30,0.15])
# Restricciones de igualdad
res3 = np.array([[1, 1, 1, 1]])
res4= np.array([1])
# Solución del problema
ans2= linprog(-fun_objt, A_ub=-res1, b_ub=-res2,  A_eq=res3, b_eq=res4,method='simplex')
pd.DataFrame(ans2.x).round(4).T*100

<ipython-input-4-c661022df4a0>:11: DeprecationWarning: `method='simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  ans2= linprog(-fun_objt, A_ub=-res1, b_ub=-res2,  A_eq=res3, b_eq=res4,method='simplex')


,0,1,2,3
0,0.0,15.79,57.89,26.32


In [ ]:
v_maximo= -ans2.fun
v_maximo*100

13.736842105263158

In [ ]:
n = 4
x = cp.Variable(n)
objetivo= cp.Minimize(-fun_objt.T@x)
restricciones = [ res1 @ x >= res2, cp.sum(x)==1, x>= 0]
prob= cp.Problem(objetivo, restricciones)
prob.solve()
ans2_= x.value
print(round(pd.DataFrame(ans2_).T,4))

     0       1       2       3
0  0.0  0.1579  0.5789  0.2632


In [ ]:
#Interpretación: Para que se llegue a un retorno máximo de 13.73% el peso para el fondo #1 debe ser del 15.79%, para el fondo 2 debe ser del 57.89% y para el fondo 3 debe ser del 26.32%

PUNTO 3. La empresa XYZ se enfrente al siguiente problema de manejo de efectivo. Las cifras
están expresadas en millones ($).
Además, se sabe que la empresa cuenta con las siguientes fuentes de recursos:

• Una línea de crédito de hasta $100 a una tasa de interés del 1% mensual.
• Puede emitir papel comercial a 90 días con un interés total del 2% por un período
de 3 meses.
• Todos los meses se pueden invertir los fondos sobrantes a una tasa de interés del
0,3% mensual. Por ejemplo, la empresa podría hacer un depósito bancario que le
reconoce esa tasa.
Sobre este problema, la empresa desea saber si: ¿usar la línea de crédito en algunos de
los meses le representa un alto costo? si es así ¿cuándo debería usarla? ¿cuánto cuesta?
¿qué pagos de intereses tendrá que hacer entre enero y junio? etc... Para responder a
estos interrogantes, la empresa puede formular y resolver un problema de LP. Para la
formulación del problema tener en cuenta lo siguiente:
Variables:
xj
: monto dispuesto a pedir en la línea de crédito en el mes j, para j = 1,2,3,4,5.
yj
: monto emitido en el papel comercial en el mes j, para j = 1,2,3.
zj
: saldo de fondos (FC) en el mes j, para j = 1,2,3,4,5,6.
Objetivo: Maximizar el saldo de los fondos (FC) en el mes 6 (Jun):

máx z6

Restricciones: Restricciones de los saldos en cada mes y límites en xj
, yj y zj
.

x1 + y1 − z1 = 150
x2 + y2 −1.01 x1 + 1.003 z1 − z2 = 100
x3 + y3 −1.01 x2 + 1.003 z2 − z3 = −200
x4 − 1.02 y1 −1.01 x3 + 1.003 z3 − z4 = 200
x5 − 1.02 y2 −1.01 x4 + 1.003 z4 − z5 = −50
−1.02 y3 −1.01 x5 + 1.003 z5 − z6 = −300

xj ≤ 100 para j = 1,2,3,4,5
xj ≥ 0 para j = 1,2,3,4,5
yj ≥ 0 para j = 1,2,3.
zj ≥ 0 para j = 1,2,3,4,5

Nota: resuelva el problema anterior usando las librerías de Scipy y CVXPY de Python.

In [18]:
x3 = np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,1])#

A3 = np.array([[1,0,0,0,0,1,0,0,-1,0,0,0,0,0],
              [-1.01,1,0,0,0,0,1,0,1.003,-1,0,0,0,0],
              [0,-1.01,1,0,0,0,0,1,0,1.003,-1,0,0,0],
              [0,0,-1.01,1,0,-1.02,0,0,0,0,1.003,-1,0,0],
              [0,0,0,-1.01,1,0,-1.02,0,0,0,0,1.003,-1,0],
              [0,0,0,0,-1.01,0,0,-1.02,0,0,0,0,1.003,-1]])

b3 = np.array([150,100,-200,200,-50,-300])

bounds3 = [(0,100),
       (0,100),
       (0,100),
       (0,100),
       (0,100),
       (0,None),
       (0,None),
       (0,None),
       (0,None),
       (0,None),
       (0,None),
       (0,None),
       (0,None),
       (0,None),]

ans3 = linprog(-x3, A_eq = A3, b_eq = b3,
              bounds = bounds3, method = 'revised simplex')

ans3.x

<ipython-input-18-4f3430f3979f>:27: DeprecationWarning: `method='revised simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  ans3 = linprog(-x3, A_eq = A3, b_eq = b3,


array([  0.        ,  50.98039216,   0.        ,   0.        ,
         0.        , 150.        ,  49.01960784, 203.43436358,
         0.        ,   0.        , 351.9441675 ,   0.        ,
         0.        ,  92.49694915])

¿Cuando deberia usarla? 

¿cuánto cuesta?

¿qué pagos de intereses tendrá que hacer entre enero y junio?

In [ ]:
#Libreria CVPXPY
n=14
x=cp.Variable(n)
objt=cp.Minimize(-x3.T@x)
restc3=[A3 @ x == b3, x >= 0]
prob= cp.Problem(objt,restc3)
prob.solve()
ans3_= x.value
print(round(pd.DataFrame(ans3_).T,2))

    0      1    2    3     4      5      6       7    8    9       10   11  \
0  0.0  42.69  0.0  0.0  8.46  150.0  57.31  195.06  0.0  0.0  351.94  0.0   

    12    13  
0  0.0  92.5  


In [ ]:
#Interpretación: Usar la línea de crédito solo es viable para el segundo mes respectivamente de $50.9803
#La línea de bono se usará en el primero, segundo y tercer mes. Con valores de: $150, $49.02 y $203,43 ; respectivamente. 
# Y finalmente, bajo las restricciones utilizadas, para el tercer mes se podrá permitir un excedente de $351,94 y un saldo máximo de $92,5 para el sexto mes.

Punto 4. La empresa XYZ desea encontrar el saldo óptimo que debe mantener en su cuenta
bancaria en un horizonte de un mes buscando maximizar su rentabilidad total. Se sabe
que la empresa espera recibir entradas de efectivo de: $500 en la primera semana, $300
en la segunda semana, $200 en la tercera semana y $400 en la cuarta semana. Todas las
cifras se encuentran en millones de pesos. Además, la empresa debe realizar los
siguientes pagos durante el mes: $350 en la primera semana, $400 en la segunda
semana, $250 en la tercera semana y $200 en la cuarta semana y, para ello, debe
mantener un saldo suficiente que respalde estas obligaciones.
Para aprovechar la rentabilidad que ofrecen los saldos en cada momento, la empresa
puede invertir sus recursos en un instrumento a corto plazo en el que recibe un
rendimiento del 0,02% por día, pero al realizar esta operación debe incurrir en un costo de
transacción del 0,01% del monto transferido.
Formule un problema de LP utilizando Scipy que le permita maximizar la rentabilidad total
de los saldos en su cuenta teniendo en cuenta las restricciones de sus flujos de efectivo.

In [ ]:
# Función objetivo
Func_objt = np.array([0,0,0,0,0,0,0,1])
# Restricciones de pago
rn=(1+0.0001)**7 #tasa de rentabilidad
A= np.array([[1, 0, 0, 0,-1,0,0,0],
             [0, 1, 0, 0,rn,-1,0,0],
             [0, 0, 1, 0,0,rn,-1,0],
             [0, 0, 0, 1,0,0,rn,-1]])
    
B= np.array([[150,-100,-50,200]])
# Funcion optimizacion

Respuesta = linprog(-Func_objt, A_ub=A,  b_ub=-B, method="simplex")
Val_opti = pd.DataFrame(Respuesta.x).T
Val_opti


<ipython-input-9-5559109acdcc>:13: DeprecationWarning: `method='simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  Respuesta = linprog(-Func_objt, A_ub=A,  b_ub=-B, method="simplex")


,0,1,2,3,4,5,6,7
0,0.0,0.0,0.0,0.0,150.0,50.105032,0.140116,200.140214


In [ ]:
#Interpretación: En los valores del 0 ,1 ,2 ,3 encontramos que es cero debido a que cada flujo se reinvirtio efectivamente en cada semana , en las columnas 4, 5, 6,7
# observamos el flujo respectivo de cada semana mas la rentabilidad de los intereses , 
#en la cuarta semana encontramos que los intereses son 0,140214 pudo haber sido mas pero en la semana 2 y 3 encontramos perdidas

5. La empresa XYZ tiene que decidir como distribuye su presupuesto $10.000 millones y la mano de obra (horas de trabajo) en tres departamentos: ventas, marketing y producción buscando maximizar sus ganancias. Para ello, se sabe que cada departamento le proporciona a la empresa las siguientes ganancias: ventas: $5, marketing: 8$, y producción: $10. Esta asignación debe cumplir las siguientes restricciones:
i. El total de horas de trabajo disponibles es de 1.000.
ii. El departamento de ventas debe recibir al menos el 20% del presupuesto total.
iii. El departamento de marketing debe recibir al menos el 30% del total de horas de trabajo disponibles.
iv. El departamento de producción debe recibir al menos el 25% del total de horas de trabajo disponibles.
Formule un problema de LP utilizando Scipy que le permita maximizar

In [31]:
x = np.array([0,0,0,5,8,10])

A5 = np.array([[0,0,0,1,0,0],
               [0,1,0,0,0,0],
               [0,0,1,0,0,0],
               [0,0,0,0,1,0],
               [0,0,0,0,0,1]])

b5 = np.array([2000,300,250,2000,2000])

A6 = np.array([[1,1,1,0,0,0],[0,0,0,1,1,1]])
b6 = np.array([1000,10000])

ans5 = linprog(-x, A_ub = -A5, b_ub=-b5, A_eq=A6, b_eq=b6, method ='simplex')
print(round(pd.DataFrame(ans5.x),2))

        0
0   450.0
1   300.0
2   250.0
3  2000.0
4  2000.0
5  6000.0


<ipython-input-31-bcc0bcd0e536>:14: DeprecationWarning: `method='simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  ans5 = linprog(-x, A_ub = -A5, b_ub=-b5, A_eq=A6, b_eq=b6, method ='simplex')


In [ ]:
#Interpretación: Para que la empresa pueda distribuir los $10.000.000 y la mano de obra en tres departamentos: ventas, marketing y producción. Con el objetivo 
#de maximizar la ganancia, la distribución se debe realizar de esta forma:
#las filas del 0 al 2 nos explica las horas distribuidas, ventas  tiene 450 horas de mano de obra ,
#marketing 300  de mano de obra y produccion 250 mano de obra
#Por otro lado, de la fila 3 a la 5: se puede observar como se van a asignar los 10millones 
#Con un resultado de: 2000 millones  a ventas , 2000 millones a marketing y 6000 millones a producción.

6. Considere una empresa con tres puntos potenciales para ubicar sus instalaciones (J =
{1,2,3}) y cinco puntos de demanda que debe satisfacer (I = {1,2,3,4,5}). La apertura del
punto j ∈ J tiene un costo fijo de funcionamiento fj, independientemente del volumen de
ventas que maneje. Este volumen está limitado a una determinada cantidad máxima que
puede manejarse anualmente (Mj). Además, hay un costo de transporte cij para ir desde el punto j para abastecer la demanda en el punto i. Las siguientes tablas muestran los datos
de demanda en cada punto:

Puntos 1 2 3 4 5
Demanda 80 270 250 160 180

Y los costos de transporte, de funcionamiento y cantidad máxima que puede mantener en
cada

Planta Costos en cada punto de demanda fj Mj

4 , 5 , 6 , 8 , 10 ,1.000 ,
500

6  4  3 5   8  1.000 500

9  7  4 3   4  1.000 500

Las columnas 1-5 muestran el costo de transporte desde cada punto instalado (planta) a
cada demanda que debe satisfacer. Las columnas fj y Mj muestran los costos fijos y la
capacidad máxima en cada ubicación. Encuentre la ubicación óptima para satisfacer las
demandas de tal forma que se minimicen los costos.
Formule un problema de optimización utilizando Scipy que le permita encontrar la
estrategia optima de la empresa para reducir sus costos de funcionamiento y transporte.

In [27]:
from cvxpy import constraints
c6 = np.array([[4, 5, 6, 8, 10], [6,4,3,5,8], [9,7,4,3,4]]) #Coeficientes función costos
b6= np.array([500,500,500]) #Coeficientes Mj
d6= np.array ( [80,270,250,160,180])
m = np.array([1000, 1000, 1000]) 

X6 = cp.Variable((3,5), nonneg=True)
Y6 = cp.Variable(3, boolean=True)


objective = cp.sum(cp.multiply(c6, X6)) + cp.sum(cp.multiply(m, Y6))
objt6=cp.Minimize(objective)

restc6=[cp.sum(X6[0,:])<=b6[0]*Y6[0],cp.sum(X6[1,:])<=b6[1]*Y6[1], cp.sum(X6[2,:])<=b6[2]*Y6[2],
                 cp.sum(X6[:,0])==d6[0],cp.sum(X6[:,1])==d6[1],cp.sum(X6[:,2])==d6[2], cp.sum(X6[:,3])==d6[3],  cp.sum(X6[:,4])==d6[4],
                 cp.sum(Y6)==2]



In [29]:
prob= cp.Problem(objt6,restc6)
prob.solve()
print(round(pd.DataFrame(X6.value),2))

      0      1      2      3      4
0  -0.0   -0.0   -0.0   -0.0   -0.0
1  80.0  270.0  150.0   -0.0   -0.0
2  -0.0   -0.0  100.0  160.0  180.0


In [ ]:
#Como se puede observar en la gráfica, para poder tener un a minimización de costos la planta cero se cierra y por lo tanto no habran costos para esta.
#Adicionalmente, si se desea que haya una estrategia optima donde se minimicen los costos, se dará la siguiente asignación por planta:
#PLANTA 1: para esta planta los costos por transpote serán de 270, los costos de funcionamiento  por 150
#PLANTA 2: para esta planta los costos de funcionamiento seran de 100, la distribución de 160 y finalmente la capcidad de 180.
#De está maneraa se respetan las restricciones de demanda, pues los resultados de las sumas de cada columna tendrán un valor total de la siguiente forma (80,270,250,160,180)